In [1]:
import os
os.getcwd()
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from collections import Counter
import random

C:\Program Files\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('Loading raw data...')
train_users_path='train_users_2.csv/train_users_2.csv'
test_users_path='test_users.csv/test_users.csv'
sessions_path='sessions.csv/sessions.csv'

Loading raw data...


## Loading data

In [3]:
#########Loading data#############
# train_users
# Load the training dataset, save the destination as target
# delete the destination from the training dataset
df_train = pd.read_csv(train_users_path)
target = df_train['country_destination']
df_train = df_train.drop(['country_destination'], axis=1)

In [4]:
df_train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome


In [5]:
# test_users
df_test = pd.read_csv(test_users_path)    
id_test = df_test['id']

In [6]:
df_test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [11]:
# sessions
# Change user_id to id
df_sessions = pd.read_csv(sessions_path)
df_sessions['id'] = df_sessions['user_id']
df_sessions = df_sessions.drop(['user_id'],axis=1)
df_sessions.head()

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NaN,NaN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NaN,NaN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NaN,NaN,Windows Desktop,435.0,d1mm9tcy42


## Preparing Session data#

#### 1. df_sessions: Fill all NaN with 'NAN' in action, action_type, device_type columns

In [12]:
print('Working on Session data...')
#Filling nan with specific value ('NAN')
df_sessions.action = df_sessions.action.fillna('NAN')
df_sessions.action_type = df_sessions.action_type.fillna('NAN')
df_sessions.action_detail = df_sessions.action_detail.fillna('NAN')
df_sessions.device_type = df_sessions.device_type.fillna('NAN')

Working on Session data...


In [13]:
df_sessions.head()

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NAN,NAN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NAN,NAN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NAN,NAN,Windows Desktop,435.0,d1mm9tcy42


#### 2. df_sessions: action modification, Action values with low frequency (set to be 100) are changed to 'OTHER'

In [14]:
df_sessions["action"].value_counts()

show                         2768278
index                         843699
search_results                725226
personalize                   706824
search                        536057
ajax_refresh_subtotal         487744
update                        365130
similar_listings              364624
social_connections            339000
reviews                       320591
active                        188036
similar_listings_v2           168788
lookup                        162041
create                        155887
dashboard                     152952
header_userpic                141830
collections                   124417
edit                          109083
campaigns                     105028
track_page_view                81117
NAN                            79626
unavailabilities               78317
qt2                            64651
notifications                  59392
confirm_email                  58726
requested                      57034
identity                       53631
a

In [15]:
# Action values with low frequency are changed to 'OTHER'
act_freq = 100  #Threshold for frequency
act = dict(zip(*np.unique(df_sessions.action, return_counts=True)))
df_sessions.action = df_sessions.action.apply(lambda x: 'OTHER' if act[x] < act_freq else x)

#### 3. df_sessions: feature engineering action_values, action_detial, action_type, device_type, section elapse

In [16]:
#Computing value_counts. These are going to be used in the one-hot encoding
#based feature generation (following loop).
f_act = df_sessions.action.value_counts().argsort() # descending
f_act_detail = df_sessions.action_detail.value_counts().argsort() # descending
f_act_type = df_sessions.action_type.value_counts().argsort() # descending
f_dev_type = df_sessions.device_type.value_counts().argsort() # descending

# grouping session by id. We will compute features from all rows with the same id.
dgr_sess = df_sessions.groupby(['id'])

In [17]:
# This is after group by what inside each iteration.
gr = list(df_sessions.groupby(['id']))[0][1]
# gr

In [18]:
f_act_detail

view_search_results               155
p3                                152
NAN                               154
-unknown-                         153
wishlist_content_update           151
user_profile                      149
change_trip_characteristics       150
similar_listings                  148
user_social_connections           147
update_listing                    146
listing_reviews                   145
dashboard                         144
user_wishlists                    143
header_userpic                    142
message_thread                    141
edit_profile                      140
message_post                      139
contact_host                      138
unavailable_dates                 137
confirm_email_link                136
create_user                       135
change_contact_host_dates         134
user_profile_content_update       133
user_reviews                      132
p5                                131
login                             130
your_trips  

In [19]:
f_act_type

view                10
data                 9
click                8
NAN                  7
-unknown-            6
submit               5
message_post         4
partner_callback     3
booking_request      2
modify               1
booking_response     0
Name: action_type, dtype: int64

In [20]:
f_dev_type

Mac Desktop                         13
Windows Desktop                     12
iPhone                              11
Android Phone                       10
iPad Tablet                          9
Android App Unknown Phone/Tablet     8
-unknown-                            7
Tablet                               6
Linux Desktop                        5
Chromebook                           4
iPodtouch                            3
Windows Phone                        2
Blackberry                           1
Opera Phone                          0
Name: device_type, dtype: int64

In [21]:
# Loop on dgr_sess to create all the features.
# This is about 10 minutes
samples = []
cont = 0
ln = len(dgr_sess)
for g in dgr_sess:
    if cont%10000 == 0:
        print("%s from %s" %(cont, ln))
    gr = g[1]
    l = []
    
    #the id, the key in groupby list
    l.append(g[0]) ##### l has the id #####
    
    #The actual first feature is the number of values. the number of actions for each users
    l.append(len(gr)) ##### l has id + the number of actions in this user #####
    
    sev = gr.secs_elapsed.fillna(0).values   # These values are used later. # these are the values secs_elapsed are known
    
    #action features
    #(how many times each value occurs, numb of unique values, mean and std)
    c_act = [0] * len(f_act) # create a list has the length of unique actions for each user
    for i,v in enumerate(gr.action.values):
        c_act[f_act[v]] += 1 # this is the user action list and has each action's count
                             # It is the reverse order of f_act
                             # In f_act, the order is show, index ....
                             # In c_act, the order is .......... index, show
    _, c_act_uqc = np.unique(gr.action.values, return_counts=True)
    c_act += [len(c_act_uqc), np.mean(c_act_uqc), np.std(c_act_uqc)]
    l = l + c_act ##### l now have 'id + number of actions in this user + 
                        # list of all actions and their count + leng of unique actions + 
                        # mean frequenc + std freq" #####
    
    #action_detail features
    #(how many times each value occurs, numb of unique values, mean and std)
    c_act_detail = [0] * len(f_act_detail)
    for i,v in enumerate(gr.action_detail.values):
        c_act_detail[f_act_detail[v]] += 1 
    _, c_act_det_uqc = np.unique(gr.action_detail.values, return_counts=True)
    c_act_detail += [len(c_act_det_uqc), np.mean(c_act_det_uqc), np.std(c_act_det_uqc)]
    l = l + c_act_detail ##### l now have 'id + number of actions in this user + 
                            # action features + action_detail features#####
    
    #action_type features
    #(how many times each value occurs, numb of unique values, mean and std
    #+ log of the sum of secs_elapsed for each value)
    l_act_type = [0] * len(f_act_type)
    c_act_type = [0] * len(f_act_type)
    for i,v in enumerate(gr.action_type.values):
        l_act_type[f_act_type[v]] += sev[i]   
        c_act_type[f_act_type[v]] += 1  
    l_act_type = np.log(1 + np.array(l_act_type)).tolist()
    _, c_act_type_uqc = np.unique(gr.action_type.values, return_counts=True)
    c_act_type += [len(c_act_type_uqc), np.mean(c_act_type_uqc), np.std(c_act_type_uqc)]
    l = l + c_act_type + l_act_type  ##### l now have 'id + number of actions in this user + 
                            # action features + action_detail features + action type + action type elapse#####  
    
    #device_type features
    #(how many times each value occurs, numb of unique values, mean and std)
    c_dev_type  = [0] * len(f_dev_type)
    for i,v in enumerate(gr.device_type .values):
        c_dev_type[f_dev_type[v]] += 1 
    c_dev_type.append(len(np.unique(gr.device_type.values)))
    _, c_dev_type_uqc = np.unique(gr.device_type.values, return_counts=True)
    c_dev_type += [len(c_dev_type_uqc), np.mean(c_dev_type_uqc), np.std(c_dev_type_uqc)]        
    l = l + c_dev_type    ##### l now have 'id + number of actions in this user + 
                            # action features + action_detail features + action type + action type elapse
                            # + device type #####  
    
    #secs_elapsed features        
    l_secs = [0] * 5 
    l_log = [0] * 15
    if len(sev) > 0:
        #Simple statistics about the secs_elapsed values.
        l_secs[0] = np.log(1 + np.sum(sev))
        l_secs[1] = np.log(1 + np.mean(sev)) 
        l_secs[2] = np.log(1 + np.std(sev))
        l_secs[3] = np.log(1 + np.median(sev))
        l_secs[4] = l_secs[0] / float(l[1])
        
        #Values are grouped in 15 intervals. Compute the number of values
        #in each interval.
        log_sev = np.log(1 + sev).astype(int)
        l_log = np.bincount(log_sev, minlength=15).tolist()                      
    l = l + l_secs + l_log 
     ##### l now have 'id + number of actions in this user + 
        # action features + action_detail features + action type + action type elapse
        # + device type + secs_elapsed simple statistiics + distribution#####  
    
    
    #The list l has the feature values of one sample.
    samples.append(l)
    cont += 1

0 from 135483
10000 from 135483
20000 from 135483
30000 from 135483
40000 from 135483
50000 from 135483
60000 from 135483
70000 from 135483
80000 from 135483
90000 from 135483
100000 from 135483
110000 from 135483
120000 from 135483
130000 from 135483


In [22]:
#Creating a dataframe with the computed features    
col_names = []    #name of the columns
for i in range(len(samples[0])-1):
    col_names.append('c_' + str(i)) 
#preparing objects    
samples = np.array(samples)
samp_ar = samples[:, 1:].astype(np.float16)
samp_id = samples[:, 0]   #The first element in obs is the id of the sample.

#creating the dataframe        
df_agg_sess = pd.DataFrame(samp_ar, columns=col_names)
df_agg_sess['id'] = samp_id
df_agg_sess.index = df_agg_sess.id

In [26]:
df_agg_sess["c_437"]

id
00023iyk9l    13.671875
0010k6l0om    13.281250
001wyh0pz8    12.554688
0028jgx1x1    12.601562
002qnbzfs5    15.687500
0031awlkjq    10.585938
0035hobuyj    15.562500
00378ocvlh    14.421875
00389675gq    14.773438
003iamz20l    14.281250
0048rkdgb1    12.703125
0057snrdpu    14.093750
005jc3nbmb    14.843750
005v5uf4dh    12.085938
0063bawn05    14.906250
006ml14zc1    12.226562
006mls2sjw    14.703125
006t3vhawl    13.609375
006z8b0uws    12.687500
0075z9e9xv    15.093750
007gj7kqdk    12.820312
009a40t3dk    14.546875
00a8dbifj8    13.117188
00allnceb8     0.000000
00an0o6c07    13.046875
00b9hfwaak    13.695312
00bn6hu437    13.726562
00bowi9sn3    11.234375
00bqkh0g5i     6.773438
00bxmflswn    15.039062
                ...    
zzq90ckj2z    13.304688
zzqb2sn066    13.273438
zzr1rkdsh9    14.617188
zzrnx9rqi5    13.968750
zzsoxtv8zb    15.500000
zzsslqqkee    12.875000
zzsszlux3t     6.882812
zzswqjsqy4    14.218750
zzt5gpsyqa    14.023438
zzt8w28nl5    14.554688
zztaeo9wd5   

In [33]:
df_agg_sess.head()
#
#c_0: the number of actions (include duplicates) for this id
#c_1 - c_231: action features
#c_232 - c_234: unique number of actions, the mean frequency of action counts, the std of action counts
#c_235 - c_390: action_detail features
#c_391 - c_393: unique number of actions detail, the mean frequency of action detail counts, the std
#c_394 - c_404: log of the sum of secs_elapsed for each value
#c_405 - c_407: unique sum of secs_elapsed for each action_type features, mean, std
#c_408 - c_415: action_type features
#c_416 - c_418: unique number of action_type features, mean, std
#c_418 - c_433: device_type features
#c_434 - c_436: unique number of device_type features, mean, std
#c_437 - c_456: secs_elapsed features        
#

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,c_448,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456,id
id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,6.0,2.0,3.0,3.0,1.0,0.0,1.0,0.0,00023iyk9l
0010k6l0om,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,12.0,2.0,8.0,4.0,3.0,0.0,0.0,0.0,0010k6l0om
001wyh0pz8,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.0,30.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,001wyh0pz8
0028jgx1x1,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,3.0,5.0,4.0,1.0,0.0,0.0,0.0,0028jgx1x1
002qnbzfs5,789.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,111.0,102.0,104.0,57.0,28.0,9.0,4.0,1.0,1.0,002qnbzfs5


In [28]:
df_agg_sess.to_csv('agg_sess.csv')

In [10]:
df_agg_sess = pd.read_csv('agg_sess.csv',index_col = 'id')

In [12]:
df_agg_sess.head()

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,c_448,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456,id.1
id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,6.0,2.0,3.0,3.0,1.0,0.0,1.0,0.0,00023iyk9l
0010k6l0om,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,12.0,2.0,8.0,4.0,3.0,0.0,0.0,0.0,0010k6l0om
001wyh0pz8,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.0,30.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,001wyh0pz8
0028jgx1x1,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,3.0,5.0,4.0,1.0,0.0,0.0,0.0,0028jgx1x1
002qnbzfs5,789.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,111.0,102.0,104.0,57.0,28.0,9.0,4.0,1.0,1.0,002qnbzfs5


In [62]:
  df_agg_sess=df_agg_sess.rename(index=str, columns={"id.1": "id"})

In [63]:
df_agg_sess.head()

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,c_448,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456,id
id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,6.0,2.0,3.0,3.0,1.0,0.0,1.0,0.0,00023iyk9l
0010k6l0om,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,12.0,2.0,8.0,4.0,3.0,0.0,0.0,0.0,0010k6l0om
001wyh0pz8,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.0,30.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,001wyh0pz8
0028jgx1x1,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,3.0,5.0,4.0,1.0,0.0,0.0,0.0,0028jgx1x1
002qnbzfs5,789.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,111.0,102.0,104.0,57.0,28.0,9.0,4.0,1.0,1.0,002qnbzfs5


#### Training & Testing Sets

#### Training set: 
0. unknown value is filled up with unknown or -1
1. the date_first_booking is dropped
2. the date account created is stored as year dac_y , month dac_m and day dac_d

In [13]:
#########Working on train and test data#####################
print('Working on users data...')
#Concatenating df_train and df_test
df_tt = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_tt.index = df_tt.id

########Creating features for train+test
#Removing date_first_booking
df_tt = df_tt.drop(['date_first_booking'], axis=1)

Working on users data...


In [14]:
#Number of missing values of every column
df_tt.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
gender                          0
age                        116866
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser                   0
dtype: int64

In [15]:
df_tt = df_tt.fillna(-1)  #Inputing this kind of missing value with -1 (missing values in train and test)
df_tt = df_tt.replace('-unknown-', -1) #-unknown is another way of missing value, then = -1.

#Number of nulls
df_tt['n_null'] = np.array([sum(r == -1) for r in df_tt.values])

In [16]:
for col in df_tt:
    print sum(df_tt[col]==-1)

0
0
0
129480
116866
0
0
1
0
0
6085
0
0
44394
0


Only consider to do something about gender,age, language, first_affiliate_tracked and first_browser.

In [17]:
Counter(df_tt.gender)

Counter({-1: 129480, 'FEMALE': 77524, 'MALE': 68209, 'OTHER': 334})

We would do nothing.

In [18]:
df_tt.language[df_tt.language==-1]

id
2a0p72u9mg    -1
Name: language, dtype: object

In [19]:
df_tt = df_tt.set_value('2a0p72u9mg', 'language', 'en')

In [20]:
Counter(df_tt.language)

Counter({'ca': 6,
         'cs': 49,
         'da': 75,
         'de': 977,
         'el': 30,
         'en': 265539,
         'es': 1174,
         'fi': 20,
         'fr': 1508,
         'hr': 2,
         'hu': 25,
         'id': 23,
         'is': 5,
         'it': 633,
         'ja': 345,
         'ko': 1116,
         'nl': 134,
         'no': 51,
         'pl': 75,
         'pt': 322,
         'ru': 508,
         'sv': 176,
         'th': 28,
         'tr': 92,
         'zh': 2634})

In [21]:
Counter(df_tt.first_affiliate_tracked)
#first_affiliate_tracked: whats the first marketing the user interacted with before the signing up

Counter({-1: 6085,
         'linked': 62064,
         'local ops': 69,
         'marketing': 281,
         'omg': 54859,
         'product': 2353,
         'tracked-other': 6655,
         'untracked': 143181})

In [22]:
#Replace all -1 with untracked
df_tt.first_affiliate_tracked = df_tt.first_affiliate_tracked.replace(-1, 'untracked')

In [23]:
Counter(df_tt.first_affiliate_tracked)

Counter({'linked': 62064,
         'local ops': 69,
         'marketing': 281,
         'omg': 54859,
         'product': 2353,
         'tracked-other': 6655,
         'untracked': 149266})

In [24]:
Counter(df_tt.first_browser)

Counter({-1: 44394,
         'AOL Explorer': 254,
         'Android Browser': 1577,
         'Apple Mail': 45,
         'Arora': 1,
         'Avant Browser': 4,
         'BlackBerry Browser': 89,
         'Camino': 9,
         'Chrome': 78671,
         'Chrome Mobile': 3186,
         'Chromium': 83,
         'CometBird': 12,
         'Comodo Dragon': 2,
         'Conkeror': 1,
         'CoolNovo': 6,
         'Crazy Browser': 2,
         'Epic': 1,
         'Firefox': 38665,
         'Flock': 2,
         'Google Earth': 1,
         'Googlebot': 1,
         'IBrowse': 1,
         'IE': 24744,
         'IE Mobile': 118,
         'IceDragon': 1,
         'IceWeasel': 14,
         'Iron': 24,
         'Kindle Browser': 1,
         'Maxthon': 60,
         'Mobile Firefox': 64,
         'Mobile Safari': 29636,
         'Mozilla': 3,
         'NetNewsWire': 1,
         'Nintendo Browser': 1,
         'OmniWeb': 2,
         'Opera': 228,
         'Opera Mini': 8,
         'Opera Mobile': 4,
  

In [28]:
# first_browser values with low frequency are changed to 'OTHER'
freq = 100  #Threshold for frequency
browser = dict(zip(*np.unique(df_tt.first_browser, return_counts=True)))
df_tt.first_browser = df_tt.first_browser.apply(lambda x: 'OTHER' if browser[x] < freq else x)

In [29]:
Counter(df_tt.first_browser)

Counter({-1: 44394,
         'AOL Explorer': 254,
         'Android Browser': 1577,
         'Chrome': 78671,
         'Chrome Mobile': 3186,
         'Firefox': 38665,
         'IE': 24744,
         'IE Mobile': 118,
         'Mobile Safari': 29636,
         'OTHER': 600,
         'Opera': 228,
         'Safari': 53302,
         'Silk': 172})

In [30]:
#Replace -1 with Other
df_tt.first_browser = df_tt.first_browser.replace(-1, 'OTHER')

In [31]:
Counter(df_tt.first_browser)

Counter({'AOL Explorer': 254,
         'Android Browser': 1577,
         'Chrome': 78671,
         'Chrome Mobile': 3186,
         'Firefox': 38665,
         'IE': 24744,
         'IE Mobile': 118,
         'Mobile Safari': 29636,
         'OTHER': 44994,
         'Opera': 228,
         'Safari': 53302,
         'Silk': 172})

In [32]:
#date_account_created
#(Computing year, month, day, week_number, weekday)
dac = np.vstack(df_tt.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df_tt['dac_y'] = dac[:,0]
df_tt['dac_m'] = dac[:,1]
df_tt['dac_d'] = dac[:,2]

In [33]:
dac_dates = [datetime(x[0],x[1],x[2]) for x in dac]

In [34]:
df_tt['dac_wn'] = np.array([d.isocalendar()[1] for d in dac_dates])

In [35]:
df_tt['dac_w'] = np.array([d.weekday() for d in dac_dates])

In [36]:
#t dummy variables
df_tt_wd = pd.get_dummies(df_tt.dac_w, prefix='dac_w')

In [37]:
df_tt = df_tt.drop(['date_account_created', 'dac_w'], axis=1)
df_tt = pd.concat((df_tt, df_tt_wd), axis=1)

In [38]:
#timestamp_first_active
#(Computing year, month, day, hour, week_number, weekday)
tfa = np.vstack(df_tt.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)

In [39]:
tfa

array([[2009,    3,   19,    4,   32,   55],
       [2009,    5,   23,   17,   48,    9],
       [2009,    6,    9,   23,   12,   47],
       ..., 
       [2014,    9,   30,   23,   54,    8],
       [2014,    9,   30,   23,   54,   30],
       [2014,    9,   30,   23,   59,    1]])

In [40]:
df_tt['tfa_y'] = tfa[:,0]
df_tt['tfa_m'] = tfa[:,1]
df_tt['tfa_d'] = tfa[:,2]
df_tt['tfa_h'] = tfa[:,3]

In [41]:
tfa_dates = [datetime(x[0],x[1],x[2],x[3],x[4],x[5]) for x in tfa]

In [42]:
df_tt['tfa_wn'] = np.array([d.isocalendar()[1] for d in tfa_dates])
df_tt['tfa_w'] = np.array([d.weekday() for d in tfa_dates])
df_tt_wd = pd.get_dummies(df_tt.tfa_w, prefix='tfa_w')
df_tt = df_tt.drop(['timestamp_first_active', 'tfa_w'], axis=1)
df_tt = pd.concat((df_tt, df_tt_wd), axis=1)

In [43]:
#timespans between dates: account created day and timestamp_first_active
#(Computing absolute number of seconds of difference between dates, sign of the difference)
df_tt['dac_tfa_secs'] = np.array([np.log(1+abs((dac_dates[i]-tfa_dates[i]).total_seconds())) for i in range(len(dac_dates))])
df_tt['sig_dac_tfa'] = np.array([np.sign((dac_dates[i]-tfa_dates[i]).total_seconds()) for i in range(len(dac_dates))])
#    df_tt['dac_tfa_days'] = np.array([np.sign((dac_dates[i]-tfa_dates[i]).days) for i in range(len(dac_dates))])

In [44]:
#Comptute seasons from dates
#(Computing the season for the two dates)
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [(0, (date(Y,  1,  1),  date(Y,  3, 20))),  #'winter'
           (1, (date(Y,  3, 21),  date(Y,  6, 20))),  #'spring'
           (2, (date(Y,  6, 21),  date(Y,  9, 22))),  #'summer'
           (3, (date(Y,  9, 23),  date(Y, 12, 20))),  #'autumn'
           (0, (date(Y, 12, 21),  date(Y, 12, 31)))]  #'winter'
def get_season(dt):
    dt = dt.date()
    dt = dt.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= dt <= end)
df_tt['season_dac'] = np.array([get_season(dt) for dt in dac_dates])
df_tt['season_tfa'] = np.array([get_season(dt) for dt in tfa_dates])
#df_all['season_dfb'] = np.array([get_season(dt) for dt in dfb_dates])

In [45]:
#Age
#(Keeping ages in 14 < age < 99 as OK and grouping others according different kinds of mistakes)
av = df_tt.age.values
av = np.where(np.logical_and(av<2000, av>1900), 2014-av, av) #This are birthdays instead of age (estimating age by doing 2014 - value)
av = np.where(np.logical_and(av<14, av>0), 14, av) #Using specific value=14 for age values below 14
av = np.where(np.logical_and(av<2016, av>2010), -1, av) #This is the current year insted of age (using specific value = -1)

In [46]:
Counter(av)

Counter({-1.0: 117615,
         14.0: 59,
         15.0: 9,
         16.0: 39,
         17.0: 81,
         18.0: 824,
         19.0: 2024,
         20.0: 896,
         21.0: 1595,
         22.0: 2508,
         23.0: 3516,
         24.0: 4571,
         25.0: 6149,
         26.0: 6745,
         27.0: 7462,
         28.0: 7687,
         29.0: 7647,
         30.0: 7712,
         31.0: 7540,
         32.0: 7187,
         33.0: 6795,
         34.0: 6176,
         35.0: 5949,
         36.0: 5032,
         37.0: 4474,
         38.0: 4091,
         39.0: 3642,
         40.0: 3327,
         41.0: 3098,
         42.0: 2731,
         43.0: 2486,
         44.0: 2584,
         45.0: 2669,
         46.0: 2318,
         47.0: 2006,
         48.0: 1779,
         49.0: 1625,
         50.0: 1679,
         51.0: 1630,
         52.0: 1459,
         53.0: 1341,
         54.0: 1248,
         55.0: 1225,
         56.0: 1130,
         57.0: 1119,
         58.0: 1005,
         59.0: 938,
         60.0: 884,
   

In [47]:
#A lot of ppl are 105, which is strange, set the value to be -1.
av = np.where(av == 105, -1, av)  #Using specific value=99 for age values above 99
av = np.where(av > 99, 99, av)  #Using specific value=99 for age values above 99

In [48]:
sum(av==-1)#Number of missing value

118966

In [49]:
av_pop = av[av>0]#Get non-missing the values of age

In [50]:
len(av_pop)

156581

In [51]:
#For the missing data of age, interpolate it according to the sample age distribution
av[av==-1]=np.random.choice(av_pop,118966,replace=True)

In [52]:
df_tt['age'] = av

In [53]:
#AgeRange
#(One-hot encoding of the edge according these intervals)
interv =  [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 100]
def get_interv_value(age):
    iv = 20
    for i in range(len(interv)):
        if age < interv[i]:
            iv = i 
            break
    return iv
df_tt['age_interv'] = df_tt.age.apply(lambda x: get_interv_value(x))
df_tt_ai = pd.get_dummies(df_tt.age_interv, prefix='age_interv')
df_tt = df_tt.drop(['age_interv'], axis=1)
df_tt = pd.concat((df_tt, df_tt_ai), axis=1)

In [54]:
#One-hot-encoding features
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_tt_dummy = pd.get_dummies(df_tt[f], prefix=f)
    df_tt = df_tt.drop([f], axis=1)
    df_tt = pd.concat((df_tt, df_tt_dummy), axis=1)   

In [55]:
df_tt.shape #We have 156 variables

(275547, 156)

In [71]:
######Merging train-test with session data#################
#Different from the original code, I keep the id column as we need for submiting data
df_all = pd.merge(df_tt, df_agg_sess, how='left')
df_all = df_all.fillna(-2)  #Missing features for samples without sesssion data.
#All types of null 
df_all['all_null'] = np.array([sum(r<0) for r in df_all.values])

In [91]:
######Computing X, y and X_test ################
piv_train = len(target) #Marker to split df_all into train + test
le = LabelEncoder()

X = df_all[:piv_train]
y = le.fit_transform(target.values)
X_test = df_all[piv_train:]
print('Shape X = %s, Shape X_test = %s'%(X.shape, X_test.shape))

Shape X = (213451, 614), Shape X_test = (62096, 614)


In [94]:
#Save out the X_test
X_test.to_csv('X_test.csv')

In [96]:
#Combine the X and y
X['y'] = y

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [100]:
X.head()

,id,age,n_null,dac_y,dac_m,dac_d,dac_wn,dac_w_0,dac_w_1,dac_w_2,...,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456,all_null,y
0,gxn3p5htnn,24.0,2,2010,6,28,26,1,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,457,7
1,820tgsjxq7,38.0,0,2011,5,25,21,0,0,1,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,457,7
2,4ft3gnwmtx,56.0,0,2010,9,28,39,0,1,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,457,10
3,bjjt8pjhuk,42.0,0,2011,12,5,49,1,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,457,11
4,87mebub9p4,41.0,1,2010,9,14,37,0,1,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,457,10


In [104]:
# Get the subset of the original X dataset, drop all users without sessions info
users_train_insession = pd.Series(list(set(X["id"]).intersection(set(df_agg_sess['id']))))
X_train = X[X['id'].isin(list(users_train_insession))]
#test

In [105]:
X_train.head()

,id,age,n_null,dac_y,dac_m,dac_d,dac_wn,dac_w_0,dac_w_1,dac_w_2,...,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456,all_null,y
137021,d1mm9tcy42,62.0,0,2014,1,1,1,0,0,1,...,15.0,4.0,9.0,10.0,14.0,5.0,1.0,0.0,1,11
137022,yo8nz8bqcq,28.0,2,2014,1,1,1,0,0,1,...,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1,7
137023,4grx6yxeby,35.0,2,2014,1,1,1,0,0,1,...,1.0,1.0,0.0,1.0,6.0,2.0,0.0,0.0,1,7
137024,ncf87guaf0,44.0,2,2014,1,1,1,0,0,1,...,28.0,29.0,7.0,6.0,7.0,7.0,1.0,0.0,1,7
137025,4rvqpxoh3h,40.0,3,2014,1,1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5


In [106]:
X_train.shape

(73815, 615)

In [108]:
X_train.to_csv('X_train.csv')